In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopandas

In [ ]:
!pip install typing_extensions==4.5.0 --upgrade
!pip install openai
!pip install tiktoken
!pip install --upgrade transformers
!pip install langchain
!pip install -U langchain-openai
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/dataset/madison/'

## Load dataset

In [ ]:
objects = pd.read_csv(path + '/input/objects_embeddings.csv', usecols=['objectid', 'geometry', 'source', 'geom_type', 'n_tokens'])
objects.head()

,objectid,geometry,source,geom_type,n_tokens
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23


In [ ]:

relations = pd.read_csv(path+'input/relations.csv', usecols=['subjectid', 'objectid', 'predicate', 'relate', 'idx'])
# relations.to_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv', index=False)
# relations = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv')
relations.head()

,objectid,subjectid,predicate,relate,idx
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0
1,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1
2,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2
3,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3
4,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4


In [ ]:
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type','n_tokens']].rename(columns={'objectid': 'subjectid',
                                                        'geometry': 'geometry_subject',
                                                        'source': 'source_subject',
                                                        'geom_type': 'geom_type_subject',
                                                          'n_tokens': 'n_tokens_subject'
                                                        }), on='subjectid', how='inner')
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type', 'n_tokens']], on='objectid', how='inner')
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24


In [ ]:
relations[(relations['geom_type_subject']=='Point')&(relations['geom_type']=='LineString')].sort_values(by=['n_tokens_subject', 'n_tokens']).iloc[0]

objectid                            road_(2633805462, 3423901632)
subjectid                                         node_3752320724
predicate                                                disjoint
relate                                                  FF0FFF102
idx                                                           195
geometry_subject                           POINT (-89.456 43.061)
source_subject                                       intersection
geom_type_subject                                           Point
n_tokens_subject                                               10
geometry             LINESTRING (-89.459 43.0661, -89.459 43.066)
source                                                       link
geom_type                                              LineString
n_tokens                                                       21
Name: 1383, dtype: object

In [ ]:
relate_df = relations[['geom_type_subject', 'predicate', 'geom_type']].drop_duplicates().reset_index(drop=True)
relate_df['rid'] = relate_df.index
relate_df['output'] = '(' + relate_df['geom_type_subject'] + ',' + relate_df['predicate'] + ',' + relate_df['geom_type'] + ')'
relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')
# relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')

In [ ]:
relations[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString       225
                            Point            225
                  crosses   LineString       225
                            Polygon          225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    LineString       225
                  overlaps  LineString       225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Point             disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Point            225
                  touches   LineString       225
                            Polygon          225
                  within    LineString       225
                            Polygon          225
Polygon           contains  LineString       225
                            Point            225
                            Polygon          225
                  crosses   LineString       225
                  disjoint  LineString       225
                            Point            225
                            Polygon          225
                  equals    Polygon          225
                  overlaps  Polygon          225
                  touches   LineString       225
                            Point            225
                            Polygon          225
                  within    Polygon          225

In [ ]:
relations[['objectid', 'predicate']].groupby(['predicate']).count()

,objectid
predicate,
contains,1125
crosses,675
disjoint,2025
equals,675
overlaps,450
touches,1800
within,1125


In [ ]:
relations.shape[0]

7875

## Train / Test split

In [ ]:
relations_example = relations[relations['idx']>=200].reset_index(drop=True)

relations_test = relations[relations['idx']<40].reset_index(drop=True)

relations_train = relations[(relations['idx']>=40)&(relations['idx']<200)].reset_index(drop=True)

In [ ]:
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,1,"(Polygon,contains,LineString)"


In [ ]:
relations_test.sort_values(by=['rid', 'idx']).head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"(LineString,contains,LineString)"
2,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1,"LINESTRING (-89.3554 43.1351, -89.3554 43.1357)",link,LineString,24,"LINESTRING (-89.3554 43.1357, -89.3554 43.1355)",link,LineString,24,0,"(LineString,contains,LineString)"
3,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2,"LINESTRING (-89.4524 43.0716, -89.4521 43.0716...",poi,LineString,57,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...",poi,LineString,46,0,"(LineString,contains,LineString)"
6,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3,"LINESTRING (-89.3392 43.0637, -89.3399 43.0641...",poi,LineString,134,"LINESTRING (-89.338 43.0659, -89.3378 43.0659)",link,LineString,23,0,"(LineString,contains,LineString)"
10,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4,"LINESTRING (-89.5168 43.0606, -89.5168 43.0616...",poi,LineString,57,"LINESTRING (-89.5175 43.0611, -89.5175 43.0607)",link,LineString,24,0,"(LineString,contains,LineString)"


In [ ]:
relations_test[['objectid', 'geom_type', 'geom_type_subject', 'predicate']].groupby(['geom_type_subject', 'predicate', 'geom_type']).count()

objectid
geom_type_subject predicate geom_type           
LineString        contains  LineString        40
                            Point             40
                  crosses   LineString        40
                            Polygon           40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    LineString        40
                  overlaps  LineString        40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Point             disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Point             40
                  touches   LineString        40
                            Polygon           40
                  within    LineString        40
                            Polygon           40
Polygon           contains  LineString        40
                            Point             40
                            Polygon           40
                  crosses   LineString        40
                  disjoint  LineString        40
                            Point             40
                            Polygon           40
                  equals    Polygon           40
                  overlaps  Polygon           40
                  touches   LineString        40
                            Point             40
                            Polygon           40
                  within    Polygon           40

## Prompts

In [ ]:
# import tiktoken, os, openai

from google.colab import userdata

KEY = userdata.get('OPENAI_QSR')
ORG = userdata.get('OPENAI_ORG')
# openai.organization = ORG
# openai.api_key = KEY
# openai.Model.list()

In [ ]:
model = 'gpt-3.5-turbo'
model_abbr = 'gpt3_5'

In [ ]:
from langchain_openai import ChatOpenAI
# process.env.OPENAI_ORGANIZATION = ORG
llm = ChatOpenAI(model_name=model, temperature=0, api_key=KEY, organization=ORG)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

### Zero-shot

#### Without dimension requirement

In [ ]:
relations_test.shape

(1400, 15)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import time

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated."""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)
preds = []

prompt_mode = 'zero_shot'
for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: (LineString, crosses, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of zero_shot, using gpt3_5.
----------------
record 100 predict: (Polygon, crosses, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of zero_shot, using gpt3_5.
----------------
record 150 predict: (Polygon, disjoint, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of zero_shot, using gpt3_5.
----------------
record 200 predict: (LineString, touches, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of zero_shot, using gpt3_5.
----------------
record 250 predict: (Polygon, contains, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of zero_shot, using gpt3_5.
----------------
record 300 predict: (Point, touches, Polygon), actual: ('Point', 'disjoint', 'Polygon')
finish sa

In [ ]:
prompt_mode = 'zero_shot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
# res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_gpt3_5,output_zero_shot_gpt3_5,rid_zero_shot_gpt3_5
36,node_3295236003,"road_(3295236002, 3295236003)",10,"(Point,touches,LineString)","(Point, equals, LineString)","(Point,equals,LineString)",-1
37,poi_way/177921886,node_3295236002,12,"(LineString,contains,Point)","(LineString, crosses, Point)","(LineString,crosses,Point)",-1
39,"road_(3297233868, 3297233867)",node_3297233868,13,"(LineString,touches,Point)","(LineString, equals, Point)","(LineString,equals,Point)",-1
47,census_550259917030,census_550250020003,18,"(Polygon,overlaps,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
48,census_550250027002,census_550250020003,18,"(Polygon,overlaps,Polygon)","(Polygon A, disjoint, Polygon B)","(Polygona,disjoint,Polygonb)",-1
...,...,...,...,...,...,...,...
1392,polygon_parcel_61010295001,parcel_61010295001,32,"(Polygon,equals,Polygon)","(Polygon A, equals, Polygon B)","(Polygona,equals,Polygonb)",-1
1393,polygon_parcel_70823109058,parcel_70823109058,32,"(Polygon,equals,Polygon)","(Polygon A, equals, Polygon B)","(Polygona,equals,Polygonb)",-1
1394,polygon_parcel_71003218031,parcel_71003218031,32,"(Polygon,equals,Polygon)","(Polygon A, equals, Polygon B)","(Polygona,equals,Polygonb)",-1
1396,polygon_parcel_71010107102,census_550250030012,19,"(Polygon,within,Polygon)","(Polygon A, disjoint, Polygon B)","(Polygona,disjoint,Polygonb)",-1


In [ ]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_gpt3_5,output_zero_shot_gpt3_5,rid_zero_shot_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_zero_shot_gpt3_5,predicate_zero_shot_gpt3_5,geom_type_zero_shot_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, overlaps, LineString)","(LineString,overlaps,LineString)",14,LineString,contains,LineString,LineString,overlaps,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, contains, LineString)","(LineString,contains,LineString)",0,LineString,contains,LineString,LineString,contains,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23,Polygon,contains,LineString,Polygon,touches,LineString


In [ ]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9971428571428571, 0.9992857142857143, 0.9964285714285714)

In [ ]:
res_valid = res #[res['rid_%s'%prompt_mode]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.627857,0.627857,0.627857,0.627857
macro avg,0.660310,0.610417,0.560875,1400.000000
weighted avg,0.679176,0.627857,0.576899,1400.000000


In [ ]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
27,26,1.000000,1.000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
22,21,1.000000,1.000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
18,17,1.000000,1.000,1.000000,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
3,2,0.973684,0.925,0.948718,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
14,13,0.944444,0.850,0.894737,40.0,LineString,touches,Point,"(LineString,touches,Point)"
13,12,0.844444,0.950,0.894118,40.0,LineString,contains,Point,"(LineString,contains,Point)"
17,16,0.860465,0.925,0.891566,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
12,11,0.942857,0.825,0.880000,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
33,32,0.770833,0.925,0.840909,40.0,Polygon,equals,Polygon,"(Polygon,equals,Polygon)"
4,3,0.864865,0.800,0.831169,40.0,LineString,disjoint,Polygon,"(LineString,disjoint,Polygon)"


#### Include dimension requirement

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import time

prompt_mode = 'zero_shot_dim'

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)
preds = []


for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of zero_shot_dim, using gpt3_5.
----------------
record 100 predict: (Polygon, crosses, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of zero_shot_dim, using gpt3_5.
----------------
record 150 predict: (Polygon, disjoint, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of zero_shot_dim, using gpt3_5.
----------------
record 200 predict: (LineString, touches, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of zero_shot_dim, using gpt3_5.
----------------
record 250 predict: (Polygon, contains, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of zero_shot_dim, using gpt3_5.
----------------
record 300 predict: (Point, touches, Polygon), actual: ('Point', 'disjoint', 

In [ ]:
prompt_mode = 'zero_shot_dim'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_dim_gpt3_5,output_zero_shot_dim_gpt3_5,rid_zero_shot_dim_gpt3_5
39,"road_(3297233868, 3297233867)",node_3297233868,13,"(LineString,touches,Point)","(LineString, equals, Point)","(LineString,equals,Point)",-1
47,census_550259917030,census_550250020003,18,"(Polygon,overlaps,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
68,node_1596592425,"road_(1596592425, 8169344295)",10,"(Point,touches,LineString)","(Point, equals, LineString)","(Point,equals,LineString)",-1
71,node_1596769289,"road_(8169344295, 1596592425)",10,"(Point,touches,LineString)","(Point, equals, LineString)","(Point,equals,LineString)",-1
77,node_6404899260,"road_(1612121520, 6404899260)",10,"(Point,touches,LineString)","(Point, equals, LineString)","(Point,equals,LineString)",-1
...,...,...,...,...,...,...,...
1328,census_550250004093,census_550250005043,31,"(Polygon,touches,Polygon)","(POLYGON, crosses, POLYGON)","(Polygon,crosses,Polygon)",-1
1347,parcel_70813329129,parcel_70813320888,31,"(Polygon,touches,Polygon)","(POLYGON, crosses, POLYGON)","(Polygon,crosses,Polygon)",-1
1371,parcel_70823121242,parcel_70823207167,31,"(Polygon,touches,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
1375,parcel_70811308088,polygon_parcel_70811308088,32,"(Polygon,equals,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1


In [ ]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_dim_gpt3_5,output_zero_shot_dim_gpt3_5,rid_zero_shot_dim_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_zero_shot_dim_gpt3_5,predicate_zero_shot_dim_gpt3_5,geom_type_zero_shot_dim_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, crosses, LineString)","(LineString,crosses,LineString)",29,LineString,contains,LineString,LineString,crosses,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, equals, LineString)","(LineString,equals,LineString)",22,LineString,contains,LineString,LineString,equals,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, crosses, LineString)","(LineString,crosses,LineString)",29,LineString,contains,LineString,LineString,crosses,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, crosses, LineString)","(LineString,crosses,LineString)",29,LineString,contains,LineString,LineString,crosses,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, crosses, LineString)","(Polygon,crosses,LineString)",27,Polygon,contains,LineString,Polygon,crosses,LineString


In [ ]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9257142857142857, 0.9985714285714286, 0.9985714285714286)

In [ ]:
res_valid = res #[res['rid_%s'%prompt_mode]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.377857,0.377857,0.377857,0.377857
macro avg,0.394013,0.367361,0.296062,1400.000000
weighted avg,0.405271,0.377857,0.304521,1400.000000


In [ ]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
22,21,0.975610,1.000,0.987654,40.0,Point,equals,Point,"(Point,equals,Point)"
27,26,1.000000,0.975,0.987342,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
33,32,0.636364,0.875,0.736842,40.0,Polygon,equals,Polygon,"(Polygon,equals,Polygon)"
23,22,0.475610,0.975,0.639344,40.0,LineString,equals,LineString,"(LineString,equals,LineString)"
34,33,1.000000,0.450,0.620690,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
12,11,1.000000,0.400,0.571429,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
25,24,0.392157,1.000,0.563380,40.0,Polygon,contains,Point,"(Polygon,contains,Point)"
9,8,0.393939,0.975,0.561151,40.0,Point,touches,Polygon,"(Point,touches,Polygon)"
18,17,1.000000,0.375,0.545455,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
21,20,0.350877,1.000,0.519481,40.0,Polygon,disjoint,Polygon,"(Polygon,disjoint,Polygon)"


### Few-shot

In [ ]:
# relations_example = relations_example[relations_example['idx']==200]

In [ ]:
relations_example['n_tokens_subject'].sum() + relations_example['n_tokens'].sum()

5701

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot'

prefix = """
You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- Return in the format (Geometry type A, PREDICATE, geometry type B), and nothing else. Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE in your output is one of the seven predicates stated.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- TRY YOUR BEST to understand how the spatial relations in the examples are determined.


[EXAMPLES]
"""

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
def get_examples(row):
    np.random.seed()
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]
    examples = (df[['geometry_subject', 'geometry', 'output']].sample(5).rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'output': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


preds = []
for idx, row in relations_test[:].iterrows():
    examples = get_examples(row)

    PROMPT = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=PROMPT)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    preds.append({'output': pred['text'], 'idx': idx, 'text': pred['text']})
    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)

----------------
record 50 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of few_shot, using gpt3_5.
----------------
record 100 predict: (Polygon,disjoint,LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of few_shot, using gpt3_5.
----------------
record 150 predict: (Polygon,disjoint,LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of few_shot, using gpt3_5.
----------------
record 200 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of few_shot, using gpt3_5.
----------------
record 250 predict: (Polygon,contains,Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of few_shot, using gpt3_5.
----------------
record 300 predict: (Point,disjoint,Polygon), actual: ('Point', 'disjoint', 'Polygon')
finish saving first

In [ ]:
prompt_mode = 'few_shot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds)
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output']
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

In [ ]:
prompt_mode = 'few_shot'

In [ ]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_cot_fewshot', 'predicate_cot_fewshot', 'geom_type_cot_fewshot']] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_cot_fewshot'] = res['geom_type_subject_cot_fewshot'].str[1:]
res['geom_type_cot_fewshot'] = res['geom_type_cot_fewshot'].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_few_shot_gpt3_5,output_few_shot_gpt3_5,rid_few_shot_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_cot_fewshot,predicate_cot_fewshot,geom_type_cot_fewshot,geom_type_subject_few_shot_gpt3_5,predicate_few_shot_gpt3_5,geom_type_few_shot_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, equals, LineString)","(LineString,equals,LineString)",22,LineString,contains,LineString,LineString,equals,LineString,LineString,equals,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString,equals,LineString)","(LineString,equals,LineString)",22,LineString,contains,LineString,LineString,equals,LineString,LineString,equals,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString,equals,LineString)","(LineString,equals,LineString)",22,LineString,contains,LineString,LineString,equals,LineString,LineString,equals,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, crosses, LineString)","(Polygon,crosses,LineString)",27,Polygon,contains,LineString,Polygon,crosses,LineString,Polygon,crosses,LineString


In [ ]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.9014285714285715, 0.9014285714285715, 0.9014285714285715)

In [ ]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.479398,0.479398,0.479398,0.479398
macro avg,0.460325,0.468791,0.409051,1262.000000
weighted avg,0.453959,0.479398,0.409699,1262.000000


In [ ]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
17,17,1.000000,1.000000,1.000000,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
26,26,1.000000,1.000000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
21,21,1.000000,1.000000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
13,13,0.781250,0.925926,0.847458,27.0,LineString,touches,Point,"(LineString,touches,Point)"
10,10,0.857143,0.818182,0.837209,22.0,Point,touches,LineString,"(Point,touches,LineString)"
2,2,0.678571,0.974359,0.800000,39.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
33,33,0.775000,0.775000,0.775000,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
32,32,0.647059,0.846154,0.733333,39.0,Polygon,equals,Polygon,"(Polygon,equals,Polygon)"
12,12,0.833333,0.588235,0.689655,17.0,LineString,contains,Point,"(LineString,contains,Point)"
24,24,0.545455,0.750000,0.631579,40.0,Polygon,contains,Point,"(Polygon,contains,Point)"


## Chain of Thought

#### Zero-shot-CoT

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain

prompt_mode = 'zero_shot_cot'

SYSTEM_PROMPT = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the [Predicate] is one of the seven predicates stated,
and the Geometry types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
"""

USER_PROMPT = """
Geometry A: {WKT_A}
Geometry B: {WKT_B}
"""

AI_PROMPT = """
Let's think step by step.
"""

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("user", USER_PROMPT),
        ("ai", AI_PROMPT),
    ],
)

chain = LLMChain(llm=llm, prompt=PROMPT)

preds = []
for idx, row in relations_test[:].iterrows():
    # prompt = PROMPT.format_prompt(WKT_A=row['geometry_subject'], WKT_B=row['geometry'])
    # print(prompt)
    # pred = llm(prompt)
    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(pred['text'])
    # print('****')

    # k = pred['text'].find('ANSWER: ')
    # preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 10 predict: First, let's visualize the geometries A and B to get a better understanding of their spatial relationship.

Geometry A: LINESTRING (-89.3392 43.0637, -89.3399 43.0641, -89.3402 43.0639, -89.3405 43.0643, -89.3391 43.0659, -89.3377 43.0659, -89.3378 43.0655, -89.3385 43.0648, -89.3376 43.0645, -89.3377 43.0644, -89.3383 43.0647, -89.3392 43.0637)

Geometry B: LINESTRING (-89.3378 43.0659, -89.3372 43.0659)

By plotting these coordinates on a map, we can see that Geometry B is a subset of Geometry A. Geometry B lies completely within Geometry A.

Now, let's analyze the spatial relationship between Geometry A and Geometry B using the DE-9IM matrix.

Geometry A: LINESTRING (-89.3392 43.0637, -89.3399 43.0641, -89.3402 43.0639, -89.3405 43.0643, -89.3391 43.0659, -89.3377 43.0659, -89.3378 43.0655, -89.3385 43.0648, -89.3376 43.0645, -89.3377 43.0644, -89.3383 43.0647, -89.3392 43.0637)

Geometry B: LINESTRING (-89.3378 43.0659, -89.3372 43.0659)

Accordi

In [ ]:
len(preds)

2800

In [ ]:
prompt_mode = 'zero_shot_cot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds).drop_duplicates(subset='idx', keep='last')
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_zero_shot_cot_gpt3_5,output_zero_shot_cot_gpt3_5,rid_zero_shot_cot_gpt3_5
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, overlaps, LineString)","(Polygon,overlaps,LineString)",-1
7,poi_way/257174438,"road_(1998162839, 1998162788)",0,"(LineString,contains,LineString)",let's visualize the geometries A and B to get ...,let'svisualizethegeometriesaandbtogetabetterun...,-1
19,parcel_70921108019,census_550250032001,20,"(Polygon,disjoint,Polygon)",let's determine the dimension of the geometrie...,let'sdeterminethedimensionofthegeometriesaandb...,-1
44,census_550250020003,"road_(1706466103, 5474511978)",27,"(Polygon,crosses,LineString)","(POLYGON, within, LINESTRING)","(Polygon,within,LineString)",-1
87,poi_way/682541525,poi_way/682541526,0,"(LineString,contains,LineString)","(LineString, does not touch, LineString)","(LineString,doesnottouch,LineString)",-1
...,...,...,...,...,...,...,...
1359,parcel_70921108019,"road_(7811575601, 7185777721)",23,"(Polygon,touches,LineString)","(POLYGON, within, LINESTRING)","(Polygon,within,LineString)",-1
1373,parcel_70826419058,census_550250004092,18,"(Polygon,overlaps,Polygon)",let's determine the dimension of the geometrie...,let'sdeterminethedimensionofthegeometriesaandb...,-1
1383,parcel_70931310183,polygon_parcel_70931310183,32,"(Polygon,equals,Polygon)",let's check if the geometries A and B are equal.,let'scheckifthegeometriesaandbareequal.,-1
1390,polygon_parcel_71005123070,parcel_71005123070,32,"(Polygon,equals,Polygon)",let's check if the two polygons are equal.,let'scheckifthetwoPolygonsareequal.,-1


In [ ]:
prompt_mode = 'zero_shot_cot'
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_zero_shot_cot_gpt3_5,output_zero_shot_cot_gpt3_5,rid_zero_shot_cot_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_cot_fewshot,predicate_cot_fewshot,geom_type_cot_fewshot,geom_type_subject_zero_shot_cot_gpt3_5,predicate_zero_shot_cot_gpt3_5,geom_type_zero_shot_cot_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString,LineString,touches,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, overlaps, LineString)","(na, na, na)",-1,Polygon,contains,LineString,na,na,na,na,na,na


In [ ]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.8914285714285715, 0.8914285714285715, 0.8914285714285715)

In [ ]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.443109,0.443109,0.443109,0.443109
macro avg,0.453677,0.438684,0.400753,1248.000000
weighted avg,0.456047,0.443109,0.404520,1248.000000


In [ ]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
21,21,0.833333,1.000000,0.909091,40.0,Point,equals,Point,"(Point,equals,Point)"
26,26,1.000000,0.764706,0.866667,34.0,Point,disjoint,Point,"(Point,disjoint,Point)"
2,2,0.739130,0.850000,0.790698,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
17,17,0.756098,0.794872,0.775000,39.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
33,33,0.658537,0.675000,0.666667,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
4,4,0.487500,1.000000,0.655462,39.0,Point,disjoint,Polygon,"(Point,disjoint,Polygon)"
10,10,0.510204,0.892857,0.649351,28.0,Point,touches,LineString,"(Point,touches,LineString)"
11,11,0.590909,0.684211,0.634146,38.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
13,13,0.509091,0.823529,0.629213,34.0,LineString,touches,Point,"(LineString,touches,Point)"
25,25,0.500000,0.562500,0.529412,32.0,Polygon,touches,Point,"(Polygon,touches,Point)"


#### Few-shot-CoT

In [ ]:
# # for _, row in relations_example.iterrows():

# def get_cot_example(row):
#     relate = list(row['relate'])
#     WKT_A = row['geometry_subject']
#     WKT_B = row['geometry']
#     Type_A = row['geom_type_subject']
#     Type_B = row['geom_type']

#     example = f"""
# First, let's determine the geometry types of A and B.
# Geometry A is a {Type_A}.
# Geometry B is a {Type_B}.

# """
#     predicate = None
#     within = False
#     contains = False

#     idx = 1
#     IA_IB = 'No' if row['predicate'] in ['disjoint', 'touches'] else 'Yes' #relate[0]=='F' else 'Yes'
#     example += f"{idx}. Does the Interior of A intersect the Interior of B? \n{IA_IB}\n"
#     idx+=1



#     if IA_IB == 'No':
#         BA_BB = 'No' if row['predicate']=='disjoint' else 'Yes' #relate[4]=='F' else 'Yes'
#         example += f"{idx}. Does A intersect B? \n{BA_BB}\n"
#         idx+=1

#         if BA_BB=='No':
#             predicate = 'disjoint'
#         else:
#             predicate = 'touches'

#         example += f"\nSo {Type_A} A {predicate} {Type_B} B.\n"


#     else:
#         IA_IB = 'Yes' if (row['predicate'] in ['within', 'equals']) else 'No'
#         example += f"{idx}. Does the intersection of A and B equal to  A? \n{IA_IB}\n"
#         idx+=1

#         if IA_IB=='Yes':

#             predicate = 'within'
#             within = True
#             example += f"So {Type_A} A is {predicate} {Type_B} B.\n"
#         if (IA_IB=='No') or (row['predicate']=='equals'):
#             IA_IB = 'Yes' if (row['predicate'] in ['contains', 'equals']) else 'No'
#             example += f"{idx}. Does the intersection of  A and  B equal to  B? \n{IA_IB}\n"
#             idx+=1

#             if IA_IB=='Yes':
#                 predicate = 'contains'
#                 contains = True
#                 example += f"So {Type_A} A {predicate} {Type_B} B.\n"
#             else:
#                 Dim = 'Yes' if (row['predicate']=='crosses') else 'No'

#                 example += f"""{idx}. Let Dim(A) be the dimension of the Interior of A,
#                 Dim(B) be the dimension of the Interior of B, Dim(AB) be the dimension of
#                 the Interior of the intersection between A and B. Does Dim(AB)=max(Dim(A), Dim(B))-1 hold? \n{Dim}\n"""
#                 idx+=1

#                 if Dim=='Yes':
#                     predicate = 'crosses'
#                 else:
#                     predicate = 'overlaps'
#                 example += f"So {Type_A} A {predicate} {Type_B} B.\n"

#     equals = 'Yes' if (within==True) and (contains==True) else 'No'

#     if (within==True) or (contains==True):
#         example += f"{idx}. Is this true? A is within B and also A contains B. \n{equals}\n"
#         idx+=1

#         if equals=='Yes':
#             predicate = 'equals'
#             example += f"So {Type_A} A {predicate} {Type_B} B.\n"


#     example += f"ANSWER: ({Type_A}, {predicate}, {Type_B})"

#     # print('---------')
#     # print(row['predicate'], predicate)
#     # print(example)

#     return example


In [ ]:
# for _, row in relations_example.iterrows():

def get_cot_example(row):
    relate = list(row['relate'])
    WKT_A = row['geometry_subject']
    WKT_B = row['geometry']
    Type_A = row['geom_type_subject']
    Type_B = row['geom_type']

    example = f"""
First, let's determine the geometry types of A and B.
Geometry A is a {Type_A}.
Geometry B is a {Type_B}.

"""
    predicate = None
    # within = False
    # contains = False

    idx = 1
    IA_IB = 'No' if row['predicate'] in ['disjoint', 'touches'] else 'Yes' #relate[0]=='F' else 'Yes'
    example += f"{idx}. Does the Interior of A intersect the Interior of B? \n{IA_IB}\n"
    idx+=1



    if IA_IB == 'No':
        BA_BB = 'No' if row['predicate']=='disjoint' else 'Yes' #relate[4]=='F' else 'Yes'
        example += f"{idx}. Does A intersect B? \n{BA_BB}\n"
        idx+=1

        if BA_BB=='No':
            predicate = 'disjoint'
        else:
            predicate = 'touches'

        example += f"\nSo {Type_A} A {predicate} {Type_B} B.\n"


    else:
        IA_IB = 'Yes' if (row['predicate'] in ['within', 'equals']) else 'No'
        example += f"{idx}. Does the intersection of A and B equal to  A? \n{IA_IB}\n"
        idx+=1

        if IA_IB=='Yes':
            contains = "No" if row['predicate']=='Within' else "Yes"
            example += f"{idx}. Does the intersection of  A and  B equal to  B? \n{contains}\n"
            idx += 1
            if contains=="No":
                predicate = 'within'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"

            else:
                predicate = 'equals'
                example += f"So {Type_A} A is {predicate} {Type_B} B.\n"


        if (IA_IB=='No'):
            IA_IB = 'Yes' if (row['predicate'] in ['contains']) else 'No'
            example += f"{idx}. Does the intersection of  A and  B equal to  B? \n{IA_IB}\n"
            idx+=1

            if IA_IB=='Yes':
                predicate = 'contains'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"
            else:
                Dim = 'No' if (row['predicate']=='crosses') else 'Yes'

                # example += f"""{idx}. Let Dim(A) be the dimension of the Interior of A,
                # Dim(B) be the dimension of the Interior of B, Dim(AB) be the dimension of
                # the Interior of the intersection of A and B. Does Dim(AB)=max(Dim(A), Dim(B))-1 hold? \n{Dim}\n"""
                example += f"""{idx}. Are A an B of the same dimension and
                  their intersection set results in a geometry different from both but of the same dimension?\n{Dim}\n"""
                idx+=1

                if Dim=='No':
                    predicate = 'crosses'
                else:
                    predicate = 'overlaps'
                example += f"So {Type_A} A {predicate} {Type_B} B.\n"


    example += f"ANSWER: ({Type_A}, {predicate}, {Type_B})"

    # print('---------')
    # print(row['predicate'], predicate)
    # print(example)

    return example


In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot_cot'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    examples = (df[['geometry_subject', 'geometry', 'example']].sample(5).rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


preds = []
for idx, row in relations_test[:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 50 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of few_shot_cot, using gpt3_5.
----------------
record 100 predict: (Polygon, touches, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of few_shot_cot, using gpt3_5.
----------------
record 150 predict: (Polygon, touches, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of few_shot_cot, using gpt3_5.
----------------
record 200 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of few_shot_cot, using gpt3_5.
----------------
record 250 predict: (Polygon, touches, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of few_shot_cot, using gpt3_5.
----------------
record 300 predict: (Point, touches, Polygon), actual: ('Point', 'disjoint', 'Polyg

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4257 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
preds = pd.read_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode)).to_dict('records')
start_idx = len(preds)
start_idx

500

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot_cot'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""

relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    if df.shape[0]>5:
        df = df.sample(5)

    examples = (df[['geometry_subject', 'geometry', 'example']].rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


# preds = []
for idx, row in relations_test.iloc[start_idx:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 550 predict: (LineString, touches, LineString), actual: ('LineString', 'within', 'LineString')
finish saving first 550 predictions of few_shot_cot, using gpt3_5.
----------------
record 600 predict: (LineString, disjoint, LineString), actual: ('LineString', 'disjoint', 'LineString')
finish saving first 600 predictions of few_shot_cot, using gpt3_5.
----------------
record 650 predict: (Point, touches, LineString), actual: ('Point', 'within', 'LineString')
finish saving first 650 predictions of few_shot_cot, using gpt3_5.
----------------
record 700 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 700 predictions of few_shot_cot, using gpt3_5.
----------------
record 750 predict: (LineString, touches, Polygon), actual: ('LineString', 'disjoint', 'Polygon')
finish saving first 750 predictions of few_shot_cot, using gpt3_5.
----------------
record 800 predict: (LineString, touches, LineString), actual: ('LineSt

In [ ]:
prompt_mode = 'few_shot_cot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds).drop_duplicates(subset='idx', keep='last')
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['text'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_few_shot_cot_gpt3_5,output_few_shot_cot_gpt3_5,rid_few_shot_cot_gpt3_5
47,census_550259917030,census_550250020003,18,"(Polygon,overlaps,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
48,census_550250027002,census_550250020003,18,"(Polygon,overlaps,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
86,parcel_60801113048,census_550250005061,19,"(Polygon,within,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
91,census_550250016041,census_550250016032,31,"(Polygon,touches,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
106,census_550250005041,census_550250004071,31,"(Polygon,touches,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
...,...,...,...,...,...,...,...
1391,parcel_71015200886,polygon_parcel_71015200886,32,"(Polygon,equals,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
1393,polygon_parcel_70823109058,parcel_70823109058,32,"(Polygon,equals,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
1395,polygon_parcel_081034401026,parcel_081034401026,32,"(Polygon,equals,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1
1396,polygon_parcel_71010107102,census_550250030012,19,"(Polygon,within,Polygon)","(Polygon, crosses, Polygon)","(Polygon,crosses,Polygon)",-1


In [ ]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_few_shot_cot_gpt3_5,output_few_shot_cot_gpt3_5,rid_few_shot_cot_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_few_shot_cot_gpt3_5,predicate_few_shot_cot_gpt3_5,geom_type_few_shot_cot_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23,Polygon,contains,LineString,Polygon,touches,LineString


In [ ]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.8921428571428571, 0.8921428571428571, 0.8921428571428571)

In [ ]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.465172,0.465172,0.465172,0.465172
macro avg,0.444419,0.449840,0.378070,1249.000000
weighted avg,0.468908,0.465172,0.394289,1249.000000


In [ ]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
21,21,1.000000,1.000000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
26,26,1.000000,1.000000,1.000000,40.0,Point,disjoint,Point,"(Point,disjoint,Point)"
17,17,1.000000,0.950000,0.974359,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
2,2,1.000000,0.750000,0.857143,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
33,33,1.000000,0.666667,0.800000,39.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
16,16,0.774194,0.600000,0.676056,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
13,13,0.487805,1.000000,0.655738,40.0,LineString,touches,Point,"(LineString,touches,Point)"
20,20,0.607143,0.708333,0.653846,24.0,Polygon,disjoint,Polygon,"(Polygon,disjoint,Polygon)"
11,11,1.000000,0.450000,0.620690,40.0,Polygon,disjoint,LineString,"(Polygon,disjoint,LineString)"
8,8,0.449438,1.000000,0.620155,40.0,Point,touches,Polygon,"(Point,touches,Polygon)"


## Multi-step

In [ ]:
# for _, row in relations_example.iterrows():

def get_cot_example(row):
    relate = list(row['relate'])
    WKT_A = row['geometry_subject']
    WKT_B = row['geometry']
    Type_A = row['geom_type_subject']
    Type_B = row['geom_type']

    example = f"""
First, let's determine the geometry types of A and B.
Geometry A is a {Type_A}.
Geometry B is a {Type_B}.

Now, let's analyze the spatial relationship between A and B.

"""

    equals = 'Yes' if row['predicate']=='equals' else 'No'
    example += f"""1. Check if A equals B.\n{equals}\n"""

    contains = 'Yes' if row['predicate']=='contains' else 'No'
    example += f"""2. Check if A contains B. \n{contains}\n"""

    within = 'Yes' if row['predicate']=='within' else 'No'
    example += f"""3. Check if A is within B. \n{within}\n"""

    touches = 'Yes' if row['predicate']=='touches' else 'No'
    example += f"""4. Check if A touches B by testing whether either of the geometries’ boundaries intersect or
    only one of the geometry’s interiors intersects the other’s boundary. \n{touches}\n"""

    crosses = 'Yes' if row['predicate']=='crosses' else 'No'
    example += f"""5. Check if A crosses B by testing whether the intersection results in a geometry whose dimension
    is one less than the maximum dimension of the two source geometries and the intersection set is interior to both
    source geometries. \n{crosses}\n"""

    overlaps = 'Yes' if row['predicate']=='overlaps' else 'No'
    example += f"""6. Check if A overlaps B by testing whether two geometries of the same dimension and
    their intersection set results in a geometry different from both but of the same dimension. \n{overlaps}\n"""

    disjoint = 'Yes' if row['predicate']=='disjoint' else 'No'
    example += f"""7. Check if A overlaps B by testing whether two geometries of the same dimension and
    their intersection set results in a geometry different from both but of the same dimension. \n{disjoint}\n"""


    example += f"ANSWER: ({Type_A}, {row['predicate']}, {Type_B})"

    # print('---------')
    # print(row['predicate'], predicate)
    # print(example)

    return example


In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot_cot_step'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    if df.shape[0]>4:
        df = df.sample(4)

    examples = (df[['geometry_subject', 'geometry', 'example']].rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


preds = []
for idx, row in relations_test[:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 50 predict: (LineString, touches, LineString), actual: ('LineString', 'contains', 'LineString')
finish saving first 50 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 100 predict: (Polygon, touches, LineString), actual: ('Polygon', 'touches', 'LineString')
finish saving first 100 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 150 predict: (Polygon, touches, LineString), actual: ('Polygon', 'disjoint', 'LineString')
finish saving first 150 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 200 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 200 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 250 predict: (Polygon, touches, Point), actual: ('Polygon', 'contains', 'Point')
finish saving first 250 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 300 predict: (Point, touches, Polygon), actual: ('P

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4176 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
preds = pd.read_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode)).to_dict('records')
start_idx = len(preds)
start_idx

450

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate

prompt_mode = 'few_shot_cot_step'

prefix = """You will be given the WKT format of geometries given the subject A and reference object B.
Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

- MAKE SURE the rationale is complete, professional, consistent, coherent, and accurate to derive the answer for the classification task.
- MAKE SURE the output ends with the 'ANSWER: (Geometry Type A, PREDICATE, Geometry Type B)', where the PREDICATE is one of the seven predicates stated,
and the Geometry Types are Point, LineString, Polygon.
- MAKE SURE the PREDICATE satisfies the dimension requirements defined by DE-9IM and OGC given Geometry Type A and Geometry Type B.
- MAKE SURE the same decision process as shown in the examples is used to reason about the answer.
- DO NOT ask repeated questions.

[EXAMPLES]
"""

# """You will be given the WKT format of geometries given the subject A and reference object B.
# Classify their spatial relations into one of the following predicates: contains, within, touches, equals, crosses, disjoint, overlaps.
# The predicates are defined by DE-9IM and Open Geospatial Consortium. If A equals to B, there is no need to return 'within' or 'contains'.

# MAKE SURE the rationale is complete.
# MAKE SURE the answer ends with the 'ANSWER: (geometry type A, predicate, geometry type B)', where the Geometry types are Point, LineString, Polygon.
# and the predicate is one of the six predicates stated. """

suffix = """
[END EXAMPLES]

Geometry A: {WKT_A}
Geometry B: {WKT_B}
Desired output:
"""
relations_example = relations_example[relations_example['idx']==200]
def get_examples(row):
    geom_type_subject = row['geom_type_subject']
    geom_type = row['geom_type']
    # create the example set
    df = relations_example[(relations_example['geom_type_subject']==geom_type_subject) &
                            (relations_example['geom_type']==geom_type)]

    df['example'] = df.apply(lambda row: get_cot_example(row), axis=1)

    if df.shape[0]>4:
        df = df.sample(4)

    examples = (df[['geometry_subject', 'geometry', 'example']].rename(
        columns={'geometry_subject': 'WKT_A',
                  'geometry': 'WKT_B',
                  'example': 'Relation'}).to_dict('records')
    )

    return examples

example_prompt = PromptTemplate(input_variables=["WKT_A", "WKT_B", 'Relation'],
                                    template="Geometry A: {WKT_A}\nGeometry B: {WKT_B}\nDesired output: {Relation}"
    )


# preds = []
for idx, row in relations_test[start_idx:].iterrows():
    examples = get_examples(row)

    prompt = FewShotPromptTemplate(
                  examples=examples,
                  example_prompt=example_prompt,
                  prefix=prefix,
                  suffix=suffix,
                  input_variables=["WKT_A", "WKT_B"],
              )

    chain = LLMChain(llm=llm, prompt=prompt)
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    # # print(pred)
    # print('predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    # pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})

    # print('-----------------------')
    # print(prompt.format(WKT_A=row['geometry_subject'], WKT_B=row['geometry']))

    # print('******')
    # print(pred['text'])

    pred = chain.invoke({'WKT_A':row['geometry_subject'], 'WKT_B': row['geometry']})
    k = pred['text'].find('ANSWER: ')
    preds.append({'text': pred['text'], 'idx': idx, 'output': pred['text'][k+8:]})

    # print('record %d'%idx, 'predict: %s, actual: %s'%(pred['text'], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
    if (idx+1)%50 == 0:
        res_llm = pd.DataFrame(preds)
        res_llm.to_csv(path+'result/temp/task1_%s_%s.csv'%(model_abbr, prompt_mode), index=False)
        print('----------------')
        print('record %d'%(idx+1), 'predict: %s, actual: %s'%(pred['text'][k+8:], (row['geom_type_subject'], row['predicate'], row['geom_type'])))
        print('finish saving first %d predictions of %s, using %s.'%(idx+1, prompt_mode, model_abbr))
        time.sleep(5)


----------------
record 500 predict: (Polygon, disjoint, Polygon), actual: ('Polygon', 'within', 'Polygon')
finish saving first 500 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 550 predict: (LineString, touches, LineString), actual: ('LineString', 'within', 'LineString')
finish saving first 550 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 600 predict: (LineString, disjoint, LineString), actual: ('LineString', 'disjoint', 'LineString')
finish saving first 600 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 650 predict: (Point, touches, LineString), actual: ('Point', 'within', 'LineString')
finish saving first 650 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 700 predict: (LineString, disjoint, Point), actual: ('LineString', 'disjoint', 'Point')
finish saving first 700 predictions of few_shot_cot_step, using gpt3_5.
----------------
record 750 predict: (LineString, touches, Polygon), a

In [150]:
# prompt_mode = 'few_shot_cot'
res = relations_test[['subjectid', 'objectid', 'rid', 'output']].copy()
res_llm = pd.DataFrame(preds).drop_duplicates(subset='idx', keep='last')
res['text_%s_%s'%(prompt_mode, model_abbr)] = res_llm['text'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res_llm['output'].values
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.lower()
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace(' ', '')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('polygon', 'Polygon')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('linestring', 'LineString')
res['output_%s_%s'%(prompt_mode, model_abbr)] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.replace('point', 'Point')

res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'output': 'output_%s_%s'%(prompt_mode, model_abbr), 'rid': 'rid_%s_%s'%(prompt_mode, model_abbr)}),
               on='output_%s_%s'%(prompt_mode, model_abbr), how='left')
res.loc[pd.isnull(res['rid_%s_%s'%(prompt_mode, model_abbr)]), 'rid_%s_%s'%(prompt_mode, model_abbr)] = -1
res['rid_%s_%s'%(prompt_mode, model_abbr)] = res['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.int32)
res.to_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr), index=False)
res[res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1]

,subjectid,objectid,rid,output,text_few_shot_cot_step_gpt3_5,output_few_shot_cot_step_gpt3_5,rid_few_shot_cot_step_gpt3_5
48,census_550250027002,census_550250020003,18,"(Polygon,overlaps,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
86,parcel_60801113048,census_550250005061,19,"(Polygon,within,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
113,parcel_70918314950,census_550250002012,20,"(Polygon,disjoint,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
140,parcel_70933202122,census_550250007003,19,"(Polygon,within,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
231,node_1221623993,node_10582096834,26,"(Point,disjoint,Point)","First, let's determine the geometry types of A...","(Point,touches,Point)",-1
276,poi_node/4479808710,poi_node/9898292787,26,"(Point,disjoint,Point)","First, let's determine the geometry types of A...","(Point,touches,Point)",-1
301,polygon_parcel_70923164506,parcel_70923143625,34,"(Polygon,contains,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
324,poi_node/2522752352,node_7216999745,26,"(Point,disjoint,Point)","First, let's determine the geometry types of A...","(Point,touches,Point)",-1
406,census_550250026032,census_550250026021,18,"(Polygon,overlaps,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1
592,parcel_70936416217,census_550250106001,20,"(Polygon,disjoint,Polygon)","First, let's determine the geometry types of A...",let'sdeterminethegeometrytypesofaandb.\ngeomet...,-1


In [144]:
res = pd.read_csv(path+'result/task1/task1_%s_%s.csv'%(prompt_mode, model_abbr))

res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
res['geom_type_subject'] = res['geom_type_subject'].str[1:]
res['geom_type'] = res['geom_type'].str[:-1]

# res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
# res[['geom_type_subject', 'predicate', 'geom_type']] = res['output'].str.split(',', expand=True)
# res['geom_type_subject'] = res['geom_type_subject'].str[1:]
# res['geom_type'] = res['geom_type'].str[:-1]

res.loc[~(res['output_%s_%s'%(prompt_mode, model_abbr)].isin(set(relate_df['output']))), 'output_%s_%s'%(prompt_mode, model_abbr)] = '(na, na, na)'
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]
res[['geom_type_subject_%s_%s'%(prompt_mode, model_abbr), 'predicate_%s_%s'%(prompt_mode, model_abbr), 'geom_type_%s_%s'%(prompt_mode, model_abbr)]] = res['output_%s_%s'%(prompt_mode, model_abbr)].str.split(',', expand=True)
res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)].str[1:]
res['geom_type_%s_%s'%(prompt_mode, model_abbr)] = res['geom_type_%s_%s'%(prompt_mode, model_abbr)].str[:-1]


res.head()

,subjectid,objectid,rid,output,text_few_shot_cot_step_gpt3_5,output_few_shot_cot_step_gpt3_5,rid_few_shot_cot_step_gpt3_5,geom_type_subject,predicate,geom_type,geom_type_subject_few_shot_cot_step_gpt3_5,predicate_few_shot_cot_step_gpt3_5,geom_type_few_shot_cot_step_gpt3_5
0,"road_(5656899646, 53496379)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
1,"road_(53496379, 5656899646)","road_(53496379, 5657142535)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
2,"road_(8997363242, 2634716954)","road_(2634716954, 8997363243)",0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
3,poi_way/1134399613,poi_way/1134399612,0,"(LineString,contains,LineString)","(LineString, touches, LineString)","(LineString,touches,LineString)",9,LineString,contains,LineString,LineString,touches,LineString
4,parcel_70920121012,poi_way/1134399612,1,"(Polygon,contains,LineString)","(Polygon, touches, LineString)","(Polygon,touches,LineString)",23,Polygon,contains,LineString,Polygon,touches,LineString


In [145]:
validity = 1 - np.sum(res['rid_%s_%s'%(prompt_mode, model_abbr)]==-1)/res.shape[0]
subject_type_match = np.sum(res['geom_type_subject']==res['geom_type_subject_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]
object_type_match = np.sum(res['geom_type']==res['geom_type_%s_%s'%(prompt_mode, model_abbr)]) / res.shape[0]

validity, subject_type_match, object_type_match

(0.99, 0.99, 0.99)

In [146]:
res_valid = res [res['rid_%s_%s'%(prompt_mode, model_abbr)]>-1]
res_valid['rid_%s_%s'%(prompt_mode, model_abbr)] = res_valid['rid_%s_%s'%(prompt_mode, model_abbr)].astype(np.float32).astype(np.int32)

In [147]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(res_valid['rid'], res_valid['rid_%s_%s'%(prompt_mode, model_abbr)], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
score

,precision,recall,f1-score,support
accuracy,0.398268,0.398268,0.398268,0.398268
macro avg,0.320712,0.399721,0.304918,1386.000000
weighted avg,0.318300,0.398268,0.302651,1386.000000


In [148]:
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
report['rid'] = report['rid'].astype(np.float32).astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report = report[report['rid']<relate_df['rid'].max()]
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
21,21,1.000000,1.000000,1.000000,40.0,Point,equals,Point,"(Point,equals,Point)"
26,26,1.000000,1.000000,1.000000,34.0,Point,disjoint,Point,"(Point,disjoint,Point)"
17,17,0.951220,0.975000,0.962963,40.0,LineString,disjoint,Point,"(LineString,disjoint,Point)"
2,2,0.857143,0.900000,0.878049,40.0,Point,disjoint,LineString,"(Point,disjoint,LineString)"
33,33,0.584615,0.950000,0.723810,40.0,Polygon,disjoint,Point,"(Polygon,disjoint,Point)"
25,25,0.592593,0.820513,0.688172,39.0,Polygon,touches,Point,"(Polygon,touches,Point)"
10,10,0.512821,1.000000,0.677966,40.0,Point,touches,LineString,"(Point,touches,LineString)"
13,13,0.506329,1.000000,0.672269,40.0,LineString,touches,Point,"(LineString,touches,Point)"
16,16,0.600000,0.525000,0.560000,40.0,LineString,disjoint,LineString,"(LineString,disjoint,LineString)"
8,8,0.390805,0.850000,0.535433,40.0,Point,touches,Polygon,"(Point,touches,Polygon)"
